In [ ]:
import csv
from tensorflow.core.example import example_pb2
from tensorflow_metadata.proto.v0 import schema_pb2

def csv_to_tfrecord(schema, csv_file, tfrecorde_file):
    reader = csv.DictReader(open(csv_file, 'r'))
    examples = []
    
    for line in reader:
        example = example_pb2.Example()
        for feature in schema.feature:
            

In [ ]:
from tensorflow_transform.tf_metadata import schema_utils

tfrecorde_file = tf.data.TFRecoedDataset(TFRECORD_DAY1)
feature_spec = schema_utils.schema_as_feature_spec(SCHEMA).feature_spec

In [ ]:
URL = 'https://storage.googleapis.com/mlep-public/course_3/week4/C3_W4_Lab_1_starter_files.tar.gz'
TAR_NAME = 'C3_W4_Lab_1_starter_files.tar.gz'
BASE_DIR = 'starter_files'
DATA_DIR = os.path.join(BASE_DIR, 'data')
CSV_DIR = os.path.join(DATA_DIR, 'csv')
TFRECORD_DIR = os.path.join(DATA_DIR, 'tfrecord')
MODELS_DIR = os.path.join(BASE_DIR, 'models')
SCHEMA_FILE = os.path.join(BASE_DIR, 'schema.pbtxt')

In [ ]:
import tensorflow as tf
import tensorflow_transform as tft

import census_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = census_constants.NUMERIC_FEATURE_KEYS
_VOCAB_FEATURE_DICT = census_constants.VOCAB_FEATURE_DICT
_BUCKET_FEATURE_DICT = census_constants.BUCKET_FEATURE_DICT
_NUM_OOV_BUCKETS = census_constants.NUM_OOV_BUCKETS
_LABEL_KEY = census_constants.LABEL_KEY

def preprocessing_fn(inputs):
    outputs = {}
    for key in _NUMERIC_FEATURE_KEYS:
        scaled = tft.scale_to_0_1(inputs[key])
        outputs[key] = tf.reshape(scaled, -1)

    for key, vocab_size in _VOCAB_FEATURE_DICT.items():
        indices = tft.compute_and_apply_vocabulary(inputs[key], num_ov_buckets=_NUM_OOV_BUCKETS)
        one_hot = tf.one_hot(indices, vocab_size + _NUM_OOV_BUCKETS)
        outputs[key] = tf.reshape(one_hot, [-1, vocab_size + _NUM_OOV_BUCKETS])

    for key, num_buckets in _BUCKET_FEATURE_DICT.items():
        indices = tft.bucketize(inputs[key], num_buckets)
        one_hot = tf.one_hot(indices, num_buckets)
        outputs[key] = tf.reshape(one_hot, [-1, num_buckets])
    outputs[_LABEL_KEY] = tf.cast(inputs[_LABEL_KEY], tf.float32)

    return outputs

In [ ]:
from typing import List, Text

import tensorflow as tf
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx.components.trainer.fn_args_utils import DataAccessor, FnArgs
from tfx_bsl.public.tfxio import TensorFlowDatasetOptions

# import same constants from transform module
import census_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = census_constants.NUMERIC_FEATURE_KEYS
_VOCAB_FEATURE_DICT = census_constants.VOCAB_FEATURE_DICT
_BUCKET_FEATURE_DICT = census_constants.BUCKET_FEATURE_DICT
_NUM_OOV_BUCKETS = census_constants.NUM_OOV_BUCKETS
_LABEL_KEY = census_constants.LABEL_KEY

def _gzip_reader_fn(filename):
    return tf.data.TFRecordDataset(filenames, compression_typr="GZIP")

def _input_fn(file_pattern,
            tf_transform_output,
            num_epochs=None
            batch_size=32) => tf.data.Dataset:
    # Get post-transform feature spec
    transformed_feature_spec = (
        tf_transform_output.transfomred_feature_spec.copy())
    # Create baches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=_LABEL_KEY
    )
    return dataset

def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        # poop label sice serving inputs do not include the label
        feature_spec.pop(_LABEL_KEY)

        # Parse raw examples into a dictionary of tensors matching the feature spec
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
         # Transoform the raw examples using the transform graph
        transformed_features = model.tft_layer(parsed_features)
        return serve_tf_examples_fn

def _build_keras_model(hidden_units: List[int] = None) -> tf.keras.Model:
    model = _wide_and_deep_classifier(
        dnn_hidden_units=hidden_units or [100, 70, 50, 25]
    )
    return model

def _wide_and_deep_classifier(dnn_hidden_units):
    # define input layers for numeric keys
    input_numeric = [
        tf.keras.layers.Input(name=colname, shape=(1,), dtype=tf.float32)
        for colname in _NUMERIC_FEATURE_KEYS
    ]
    # define input laysers for vocab kays
    input_categorical = [
        tf.keras.layers.Input(name=colname, shape=(vocab_size + _NUM_OOV_BUCKETS,), dtype=tf.float32)
        for colname, vocab_size in _VOCAB_FEATURE_DICT.items()
    ]
    
    # define input layers for bucketkey
    input_categorical += [
        tf.keras.layers.Input(name=colname, shape=(num_buckets,), dtype=tf.float32)
        for colname, num_buckets in _BUCKET_FEATURE_DICT.items()
    ]

    deep = tf.keras.laysers.concatinate(input_numeric)

    for numnodes in dnn_hidden_units:
        deep = tf.keras.layers.Dense(numnodes)(deep)
    # concatinate categorical inputs
    wide = tf.keras.layers.concatinate(input_categorical)
    wide = tf.keras.Dense(128, activation='relu')(wide)

    combined = tf.keras.layers.concatinate([deep, wide])

    output = tf.keras.layers.concatinate([deep, wide])

    input_layers = input_numeric + input_categorical
    model = tf.keras.Model(input_layers, output)

    model.compile(
        loss=''
    )




In [ ]:
# Define Constants
ATTR_KEY = "attributes"
IMAGE_KEY = "image"
LABEL_KEY = "Smiling"
GROUP_KEY = "Young"
IMAGE_SIZE = 28

def preprocess_image_dict(feat_dict):
    image = feat_dict[IMAGE_KEY]
    label = feat_dict[ATTR_KEY][LABEL_KEY]
    group = feat_dict[ATTR_KEY][GROUP_KEY]

    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    image /= 255.0

    label = tf.cast(label, tf.float32)
    group = tf.cast(group, tf.float32)

    feat_dict[IMAGE_KEY] = image
    feat_dict[ATTR_KEY][LABEL_KEY] = label
    feat_dict[ATTR_KEY][GROUP_KEY] = group

    return feat_dict


def celeb_a_train_data_wo_group(data, batch_size):
    celeb_a_train_data = data.shuffle(1024).repeat().batch(batch_size).map(preprocess_image_dict)
    return celeb_a_train_data.map(get_image_label)